<a href="https://colab.research.google.com/github/cezarviana/fake-news-no-more/blob/main/fake_news_no_more_(verificador_de_noticias).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [83]:
%pip -q install google-genai

In [82]:
# Configura a API Key do Google Gemini

import os
from google.colab import userdata

os.environ["GOOGLE_API_KEY"] = userdata.get('GOOGLE_API_KEY')

In [59]:
# Configura o cliente da SDK do Gemini

from google import genai

client = genai.Client()

MODEL_ID = "gemini-2.0-flash"

In [61]:
# Pergunta ao Gemini uma informação mais recente que seu conhecimento

from IPython.display import HTML, Markdown

# Perguntar pro modelo quando é a próxima imersão de IA ###############################################
resposta = client.models.generate_content(
    model=MODEL_ID,
    contents='Quando é a próxima Imersão IA com Google Gemini da Alura?',
)

# Exibe a resposta na tela
display(Markdown(f"Resposta:\n {resposta.text}"))

Resposta:
 A Alura não tem uma data fixa para a Imersão IA com Google Gemini. A melhor forma de saber quando será a próxima edição é:

*   **Acompanhar as redes sociais da Alura:** Fique de olho no Instagram, LinkedIn e outros canais onde a Alura costuma divulgar seus eventos e cursos.
*   **Verificar a página de Imersões da Alura:** Procure na página de Imersões da Alura (geralmente no site principal) se há alguma Imersão IA com Google Gemini programada.
*   **Assinar a newsletter da Alura:** Assim você recebe informações sobre os próximos cursos e eventos diretamente no seu e-mail.

In [62]:
# Pergunta ao Gemini uma informação utilizando a busca do Google como contexto

response = client.models.generate_content(
    model=MODEL_ID,
    contents='Quando é a próxima Imersão IA com Google Gemini da Alura?',
    config={"tools": [{"google_search": {}}]}
)

# Exibe a resposta na tela
display(Markdown(f"Resposta:\n {response.text}"))

Resposta:
 A próxima Imersão IA com Google Gemini da Alura acontecerá de 12 a 16 de maio de 2025. As inscrições estão abertas até 11 de maio de 2025. O curso é online, gratuito e não exige pré-requisitos. Ao final, você receberá um certificado Alura + Google e poderá concorrer a prêmios.

In [63]:
# Exibe a busca
print(f"Busca realizada: {response.candidates[0].grounding_metadata.web_search_queries}")
# Exibe as URLs nas quais ele se baseou
print(f"Páginas utilizadas na resposta: {', '.join([site.web.title for site in response.candidates[0].grounding_metadata.grounding_chunks])}")
print()
display(HTML(response.candidates[0].grounding_metadata.search_entry_point.rendered_content))

Busca realizada: ['próxima Imersão IA com Google Gemini da Alura', 'Alura Imersão IA Google Gemini']
Páginas utilizadas na resposta: thallesbenicio.com.br, tiktok.com, starten.tech, rodrigostoledo.com, youtube.com, alura.com.br



In [78]:
# Instalar Framework de agentes do Google ################################################
!pip install -q google-adk

In [80]:
from google.adk.agents import Agent
from google.adk.runners import Runner
from google.adk.sessions import InMemorySessionService
from google.adk.tools import google_search
from google.genai import types  # Para criar conteúdos (Content e Part)
from datetime import date
import textwrap # Para formatar melhor a saída de texto
from IPython.display import display, Markdown # Para exibir texto formatado no Colab
import requests # Para fazer requisições HTTP
import warnings

warnings.filterwarnings("ignore")

In [67]:
# Função auxiliar que envia uma mensagem para um agente via Runner e retorna a resposta final
def call_agent(agent: Agent, message_text: str) -> str:
    # Cria um serviço de sessão em memória
    session_service = InMemorySessionService()
    # Cria uma nova sessão (você pode personalizar os IDs conforme necessário)
    session = session_service.create_session(app_name=agent.name, user_id="user1", session_id="session1")
    # Cria um Runner para o agente
    runner = Runner(agent=agent, app_name=agent.name, session_service=session_service)
    # Cria o conteúdo da mensagem de entrada
    content = types.Content(role="user", parts=[types.Part(text=message_text)])

    final_response = ""
    # Itera assincronamente pelos eventos retornados durante a execução do agente
    for event in runner.run(user_id="user1", session_id="session1", new_message=content):
        if event.is_final_response():
          for part in event.content.parts:
            if part.text is not None:
              final_response += part.text
              final_response += "\n"
    return final_response

In [68]:
# Função auxiliar para exibir texto formatado em Markdown no Colab
def to_markdown(text):
  text = text.replace('•', '  *')
  return Markdown(textwrap.indent(text, '> ', predicate=lambda _: True))

In [69]:
##########################################
# --- Agente 1: Buscador de Notícias --- #
##########################################
def agente_buscador(topico, data_de_hoje):
    buscador = Agent(
        name="agente_buscador",
        model="gemini-2.0-flash",
        instruction="""
        Você é parte de um sistema colaborativo de verificação de notícias. Siga rigorosamente as instruções específicas da sua função para analisar o tópico fornecido e contribuir para a determinação da sua veracidade.
        Sua função é ser um assistente de pesquisa. A sua tarefa é usar a ferramenta de busca google (google_search) para recuperar as últimas notícias de lançamentos muito relevantes sobre o tópico abaixo.
        Priorize fontes jornalísticas reconhecidas e com boa reputação.
        Selecione no máximo 5 lançamentos que demonstrem relevância (baseada na cobertura e qualidade da fonte) e sejam os mais atuais possíveis.
        Se o tópico não possuir 5 notícias a seu respeito, apresente somente as encontradas. Sem adicionar outras notícias com termos parecidos e que não tenham relação direta com o tópico.
        Se o tópico gerar pouca cobertura noticiosa ou reações limitadas, sinalize essa baixa relevância como um possível indicativo para considerar outros tópicos.
        Esses lançamentos relevantes devem ser atuais, de no máximo um mês antes da data de hoje.
        """,
        description="Agente que busca de notícias no Google Search sobre o tópico",
        tools=[google_search]
    )

    entrada_do_agente_buscador = f"Tópico: {topico}\nData de hoje: {data_de_hoje}"
    # Executa o agente
    lancamentos_buscados = call_agent(buscador, entrada_do_agente_buscador)
    return lancamentos_buscados

In [71]:
################################################
# --- Agente 2: Verificador de Fontes --- #
################################################
def agente_verificador_fontes(topico, lancamentos_buscados):
    verificador_fontes = Agent(
        name="agente_verificador_fontes",
        model="gemini-2.0-flash",
        # Inserir as instruções do Agente Verificador de Fontes #################################################
        instruction="""
        Você é parte de um sistema colaborativo de verificação de notícias. Siga rigorosamente as instruções específicas da sua função para analisar o tópico fornecido e contribuir para a determinação da sua veracidade.
        Sua função é ser um verificador de fontes, especialista em fact-checking.
        Para cada fonte principal identificada:
        - Determine se o site ou canal é conhecido e geralmente considerado confiável.
        - Localize e examine a seção 'Sobre nós' (ou equivalente) para entender a missão, equipe e possíveis vieses do site.
        - Verifique se outras fontes confiáveis corroboram as informações apresentadas pela fonte principal. Liste as fontes que confirmam os achados.
        """,
        description="Agente que verifica as fontes levantadas",
        tools=[google_search]
    )

    entrada_do_agente_verificador_fontes = f"Tópico:{topico}\nLançamentos buscados: {lancamentos_buscados}"
    # Executa o agente
    verificacao_fontes = call_agent(verificador_fontes, entrada_do_agente_verificador_fontes)
    return verificacao_fontes

In [72]:
################################################
# --- Agente 3: Verificador de Conteúdo --- #
################################################
def agente_verificador_conteudo(topico, lancamentos_buscados):
    verificador_conteudo = Agent(
        name="agente_verificador_conteudo",
        model="gemini-2.0-flash",
        # Inserir as instruções do Agente Verificador de Conteúdo #################################################
        instruction="""
        Você é parte de um sistema colaborativo de verificação de notícias. Siga rigorosamente as instruções específicas da sua função para analisar o tópico fornecido e contribuir para a determinação da sua veracidade.
        Sua função é ser um verificador de conteúdo, especialista em fact-checking.
        Examine o conteúdo das notícias e informações relacionadas ao tópico.
        Linguagem e Estilo:
        - Avalie se há uso de sofismos ou outras técnicas de persuasão manipuladoras.
        - Identifique erros de ortografia e gramática que possam indicar falta de profissionalismo ou revisão.
        - Distinga claramente entre fatos apresentados e opiniões, verificando se as opiniões são devidamente atribuídas.
        Contexto e Evidências:
        - Verifique se a data da informação é relevante para o contexto atual.
        - Se houver imagens, utilize a busca reversa (Google Imagens) para verificar sua autenticidade e se foram usadas em outros contextos enganosos.
        Coerência:
        - Avalie a lógica interna do conteúdo e sua coerência com informações de outras fontes.
        """,
        description="Agente que verifica o conteúdo das notícias levantadas",
        tools=[google_search]
    )

    entrada_do_agente_verificador_conteudo = f"Tópico:{topico}\nLançamentos buscados: {lancamentos_buscados}"
    # Executa o agente
    verificacao_conteudo = call_agent(verificador_conteudo, entrada_do_agente_verificador_conteudo)
    return verificacao_conteudo

In [75]:
##########################################
# --- Agente 4: Agência de Fact-Checking --- #
##########################################
def agente_verificador_fatos(topico, data_de_hoje):
    verificador_fatos = Agent(
        name="agente_verificador_fatos",
        model="gemini-2.0-flash",
        instruction="""
        Você é parte de um sistema colaborativo de verificação de notícias. Siga rigorosamente as instruções específicas da sua função para analisar o tópico fornecido e contribuir para a determinação da sua veracidade.
        Sua tarefa é verificar se o tópico/afirmação já foi alvo de checagem por agências de fact-checking confiáveis, através da  busca google (google_search), priorizando as verificações mais recentes sobre o tópico.
        As agências de fact-checking a serem consultadas de acordo com o tópico serão as indicadas abaixo:
        - No Brasil:
          - Aos Fatos;
          - Lupa;
          - UOL Confere;
          - Estadão Verifica;
          - Fato ou Fake (G1);
          - Boatos.org;
          - Agência Pública - Truco no Congresso;
          - Comprova;
          - E-farsas;
          - É isso Mesmo? (O Globo);
          - Portal EBC - Checagem;
        - Internacionais com atuação ou relevância no Brasil:
          - AFP Fact Check;
          - Reuters Fact Check;
          - Snopes;
          - PolitiFact;
          - FactCheck.org;
        - Organizações e Redes Internacionais:
          - International Fact-Checking Network (IFCN);
          - European Fact-Checking Standards Network (EFCSN);
          - Duke Reporters' Lab;
        Informe se o tópico foi encontrado em alguma das agências de fact-checking e qual foi a conclusão dessas agências (verdadeiro, falso, enganoso, etc.).
        Se encontrado, cite a fonte da agência de fact-checking e um breve resumo da sua análise.
        """,
        description="Agente que verifica o que agências de fact-checking dizem a respeito do tópico",
        tools=[google_search]
    )

    entrada_do_agente_verificador_fatos = f"Tópico: {topico}\nData de hoje: {data_de_hoje}"
    # Executa o agente
    verificacao_fatos = call_agent(verificador_fatos, entrada_do_agente_verificador_fatos)
    return verificacao_fatos

In [76]:
################################################
# --- Agente 5: Organizador do Resultado da Verificação --- #
################################################
def agente_organizador_resultado(topico, lancamentos_buscados, verificacao_fontes, verificacao_conteudo, verificacao_fatos):
    organizador = Agent(
        name="agente_organizador_resultado",
        model="gemini-2.0-flash",
        # Inserir as instruções do Agente de Resultados #################################################
        instruction="""
        Você é parte de um sistema colaborativo de verificação de notícias. Siga rigorosamente as instruções específicas da sua função para analisar o tópico fornecido e contribuir para a determinação da sua veracidade.
        Sua função é organizar os resultados com base nas análises dos outros agentes sobre o tópico, e determinar a veracidade da informação.
        Apresente um veredicto claro: Verdadeiro, Falso, Enganoso, Insustentável, etc.
        Justifique sua conclusão de forma concisa, utilizando as evidências e os resultados fornecidos pelos outros agentes (agente_buscador, agente_verificador_fontes, agente_verificador_conteudo e agente_verificador_fatos).
        Liste as fontes mais relevantes (sites de notícias confiáveis, agências de fact-checking) que sustentam sua conclusão.
        """,
        description="Agente que organiza os resultados da verificação",
        tools=[google_search]
    )

    entrada_do_agente_organizador_resultado = f"Tópico:{topico}\nLançamentos buscados:{lancamentos_buscados}\nVerificação fontes:{verificacao_fontes}\nVerificação conteúdo:{verificacao_conteudo}\nVerificação fatos:{verificacao_fatos}"
    # Executa o agente
    resultado = call_agent(organizador,  entrada_do_agente_organizador_resultado)
    return resultado

In [77]:

data_de_hoje = date.today().strftime("%d/%m/%Y")

print("🚀 Iniciando o Sistema de Verificação de Fatos com 5 Agentes 🚀")

# --- Obter o Tópico do Usuário ---
topico = input("❓ Por favor, digite o TÓPICO sobre o qual você quer saber a veracidade: ")

# Inserir lógica do sistema de agentes ################################################
if not topico:
    print("Você esqueceu de digitar o tópico!")
else:
    print(f"Maravilha! Vamos pesquisar sobre a veracidade a respeito de {topico}")

    lancamentos_buscados = agente_buscador(topico, data_de_hoje)
    print("\n--- 📝 Resultado do Agente 1 (Buscador de Notícias) ---\n")
    display(to_markdown(lancamentos_buscados))
    print("--------------------------------------------------------------")

    verificacao_fontes = agente_verificador_fontes(topico, lancamentos_buscados)
    print("\n--- 📝 Resultado do Agente 2 (Verificador de Fontes) ---\n")
    display(to_markdown(verificacao_fontes))
    print("--------------------------------------------------------------")

    verificacao_conteudo = agente_verificador_conteudo(topico, lancamentos_buscados)
    print("\n--- 📝 Resultado do Agente 3 (Verificador de Conteúdo) ---\n")
    display(to_markdown(verificacao_conteudo))
    print("--------------------------------------------------------------")

    verificacao_fatos = agente_verificador_fatos(topico, data_de_hoje)
    print("\n--- 📝 Resultado do Agente 4 (Agências de Fact-Checking) ---\n")
    display(to_markdown(verificacao_fatos))
    print("--------------------------------------------------------------")

    resultado = agente_organizador_resultado(topico, lancamentos_buscados, verificacao_fontes, verificacao_conteudo, verificacao_fatos)
    print("\n--- 📝 Resultado do Agente 5 (Organizador do Resultado da Verificação) ---\n")
    display(to_markdown(resultado))
    print("--------------------------------------------------------------")

🚀 Iniciando o Sistema de Verificação de Fatos com 5 Agentes 🚀
❓ Por favor, digite o TÓPICO sobre o qual você quer saber a veracidade: terra plana
Maravilha! Vamos pesquisar sobre a veracidade a respeito de terra plana


Exception in thread Thread-105 (_asyncio_thread_main):
Traceback (most recent call last):
  File "/usr/lib/python3.11/threading.py", line 1045, in _bootstrap_inner
    self.run()
  File "/usr/lib/python3.11/threading.py", line 982, in run
    self._target(*self._args, **self._kwargs)
  File "/usr/local/lib/python3.11/dist-packages/google/adk/runners.py", line 138, in _asyncio_thread_main
    asyncio.run(_invoke_run_async())
  File "/usr/lib/python3.11/asyncio/runners.py", line 190, in run
    return runner.run(main)
           ^^^^^^^^^^^^^^^^
  File "/usr/lib/python3.11/asyncio/runners.py", line 118, in run
    return self._loop.run_until_complete(task)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/lib/python3.11/asyncio/base_events.py", line 654, in run_until_complete
    return future.result()
           ^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/google/adk/runners.py", line 126, in _invoke_run_async
    async for event in self.run_async(
  File "/


--- 📝 Resultado do Agente 1 (Buscador de Notícias) ---



--------------------------------------------------------------


Exception in thread Thread-107 (_asyncio_thread_main):
Traceback (most recent call last):
  File "/usr/lib/python3.11/threading.py", line 1045, in _bootstrap_inner
    self.run()
  File "/usr/lib/python3.11/threading.py", line 982, in run
    self._target(*self._args, **self._kwargs)
  File "/usr/local/lib/python3.11/dist-packages/google/adk/runners.py", line 138, in _asyncio_thread_main
    asyncio.run(_invoke_run_async())
  File "/usr/lib/python3.11/asyncio/runners.py", line 190, in run
    return runner.run(main)
           ^^^^^^^^^^^^^^^^
  File "/usr/lib/python3.11/asyncio/runners.py", line 118, in run
    return self._loop.run_until_complete(task)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/lib/python3.11/asyncio/base_events.py", line 654, in run_until_complete
    return future.result()
           ^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/google/adk/runners.py", line 126, in _invoke_run_async
    async for event in self.run_async(
  File "/

...

ERROR:asyncio:Task exception was never retrieved
future: <Task finished name='Task-313' coro=<AsyncClient.aclose() done, defined at /usr/local/lib/python3.11/dist-packages/httpx/_client.py:1978> exception=RuntimeError('Event loop is closed')>
Traceback (most recent call last):
  File "/usr/local/lib/python3.11/dist-packages/httpx/_client.py", line 1985, in aclose
    await self._transport.aclose()
  File "/usr/local/lib/python3.11/dist-packages/httpx/_transports/default.py", line 406, in aclose
    await self._pool.aclose()
  File "/usr/local/lib/python3.11/dist-packages/httpcore/_async/connection_pool.py", line 353, in aclose
    await self._close_connections(closing_connections)
  File "/usr/local/lib/python3.11/dist-packages/httpcore/_async/connection_pool.py", line 345, in _close_connections
    await connection.aclose()
  File "/usr/local/lib/python3.11/dist-packages/httpcore/_async/connection.py", line 173, in aclose
    await self._connection.aclose()
  File "/usr/local/lib/pyth

--------------------------------------------------------------


Exception in thread Thread-113 (_asyncio_thread_main):
Traceback (most recent call last):
  File "/usr/lib/python3.11/threading.py", line 1045, in _bootstrap_inner
    self.run()
  File "/usr/lib/python3.11/threading.py", line 982, in run
    self._target(*self._args, **self._kwargs)
  File "/usr/local/lib/python3.11/dist-packages/google/adk/runners.py", line 138, in _asyncio_thread_main
    asyncio.run(_invoke_run_async())
  File "/usr/lib/python3.11/asyncio/runners.py", line 190, in run
    return runner.run(main)
           ^^^^^^^^^^^^^^^^
  File "/usr/lib/python3.11/asyncio/runners.py", line 118, in run
    return self._loop.run_until_complete(task)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/lib/python3.11/asyncio/base_events.py", line 654, in run_until_complete
    return future.result()
           ^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/google/adk/runners.py", line 126, in _invoke_run_async
    async for event in self.run_async(
  File "/


--- 📝 Resultado do Agente 5 (Organizador do Resultado da Verificação) ---



--------------------------------------------------------------
